### Import Modules

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import arcpy
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import CoastalVarExtractor.functions_warcpy as fwa
import CoastalVarExtractor.functions as fun

### Initialize Variables

In [2]:
from CoastalVarExtractor.setvars import *

extendedTrans = os.path.join(home, 'fiis_trans')
extTrans_tidy = os.path.join(home, 'tidyTrans')

inletLines = os.path.join(home, 'inletLines')
ShorelinePts = os.path.join(home, 'SLpts_utm')
dlPts = os.path.join(home, 'DLpts_utm')
dhPts = os.path.join(home, 'DHpts_utm')
armorLines = os.path.join(home, 'armorLines')
elevGrid = os.path.join(home, 'DEM')
elevGrid_5m = os.path.join(home, 'DEM_5m')
barrierBoundary = os.path.join(home, 'bndpoly_2sl')  
shoreline = os.path.join(home, 'ShoreBetweenInlets')

SubType = os.path.join(home, 'FI12_SubType')
VegType = os.path.join(home, 'FI12_VegType')
VegDens = os.path.join(home, 'FI12_VegDen')
GeoSet = os.path.join(home, 'FI12_GeoSet')

site: FireIsland
year: 2012
setvars.py initialized variables.


### Merge from excel file

In [3]:
# 1. Import data from xls 
# File name
xls_path = r"\\IGSAGIEGGS-CSGG\Thieler_Group\Commons_DeepDive\DeepDive\NewYork\AnthropogenicData"
xls_name = "FI2012_trans_5mPtsUpCombinedsubFixALL2.xlsx"

xltbl = pd.read_excel(os.path.join(xls_path, xls_name))

In [4]:
xltbl.head()

,OBJECTID_1,OBJECTID,BASELINEID,TRANSORDER,PROCTIME,AUTOGEN,ENDX,ENDY,AZIMUTH,SHAPE_LENG,...,Dist_MHWbay,SplitSort,DistSegDH,DistSegDL,DistSegArm,PointZ,PointSlp,Development,Nourishment,Construction
0,1,3519,-99999,1,-99999.0,-99999,-99999.0,-99999.0,358.75,-99999.0,...,-99999.0,1,-99999.0,-99999.0,-99999.0,0.91820,9.842591,111,111,111
1,2,3519,-99999,1,-99999.0,-99999,-99999.0,-99999.0,358.75,-99999.0,...,-99999.0,2,-99999.0,-99999.0,-99999.0,1.19412,7.487503,111,111,111
2,3,3519,-99999,1,-99999.0,-99999,-99999.0,-99999.0,358.75,-99999.0,...,-99999.0,3,-99999.0,-99999.0,-99999.0,0.85836,5.800764,111,111,111
3,4,3519,-99999,1,-99999.0,-99999,-99999.0,-99999.0,358.75,-99999.0,...,-99999.0,4,-99999.0,-99999.0,-99999.0,1.05932,6.581713,111,111,111
4,5,3519,-99999,1,-99999.0,-99999,-99999.0,-99999.0,358.75,-99999.0,...,-99999.0,5,-99999.0,-99999.0,-99999.0,0.88016,10.083154,111,111,111


In [5]:
xltbl.columns

Index(['OBJECTID_1', 'OBJECTID', 'BASELINEID', 'TRANSORDER', 'PROCTIME',
       'AUTOGEN', 'ENDX', 'ENDY', 'AZIMUTH', 'SHAPE_LENG', 'TRANSECTID', 'LRR',
       'LR2', 'LSE', 'LCI90', 'TID2', 'LRR2', 'LR22', 'LSE2', 'LCI902',
       'SL_Lon', 'SL_Lat', 'SL_easting', 'SL_northing', 'Bslope', 'Arm_Lon',
       'Arm_Lat', 'Arm_easting', 'Arm_northing', 'Arm_z', 'DH_Lon', 'DH_Lat',
       'DH_easting', 'DH_northing', 'DH_z', 'DL_Lon', 'DL_Lat', 'DL_easting',
       'DL_northing', 'DL_z', 'DL_zMHW', 'DH_zMHW', 'Arm_zMHW', 'DistDH',
       'DistDL', 'DistArm', 'Dist2Inlet', 'WidthLand', 'WidthFull',
       'WidthPart', 'Source_beachwidth', 'MLW_easting', 'MLW_northing',
       'beach_h_MLW', 'beachWidth_MLW', 'ORIG_OID', 'seg_y', 'seg_x',
       'Dist_Seg', 'Dist_MHWbay', 'SplitSort', 'DistSegDH', 'DistSegDL',
       'DistSegArm', 'PointZ', 'PointSlp', 'Development', 'Nourishment',
       'Construction'],
      dtype='object')

Join anthro data to transects

1. Convert xls spreadsheet to points 
2. Select the first points along each transects and create new FC
3. Spatial Join the new FC to the updated transects 
    - one to one
    - keep all target features
    - keep only the ID fields and the three anthro fields (and the transect fields [LRR, etc.]?)
    - intersect

4. Join the transect values to the pts based on sort_ID


In [6]:
tr_w_anthro = os.path.join(arcpy.env.workspace, 'fiis_trans_wAnthro')

In [8]:
trans_df2 = fwa.FCtoDF(tr_w_anthro, id_fld=tID_fld, dffields=['Development', 'Nourishment','Construction'])

Converting feature class to array...
Converting array to dataframe...


In [12]:
# Get existing DFs
pts_df = pd.read_pickle(os.path.join(scratch_dir, transPts_null+'.pkl'))
trans_df = pd.read_pickle(os.path.join(scratch_dir, extTrans_null+'.pkl'))

# Join anthro fields to trans and points DFs
trans_df = fun.join_columns(trans_df, trans_df2) 
pts_df = fun.join_columns(pts_df, trans_df, tID_fld)

# Save dataframes to open elsewhere or later
trans_df.to_pickle(os.path.join(scratch_dir, extTrans_null+'_anthro.pkl'))
pts_df.to_pickle(os.path.join(scratch_dir, transPts_null+'_anthro.pkl'))

,SplitSort,GeoSet,seg_x,seg_y,SubType,VegDens,VegType,ptSlp,ptZ,sort_ID,...,ub_feat,Dist2Inlet,WidthFull,WidthLand,WidthPart,mean_Zmhw,max_Zmhw,Construction,Development,Nourishment
0,0,1.0,642131.5400,4.498927e+06,4444.0,111.0,11.0,4.445196,0.68200,1,...,NaN,NaN,65.098101,65.098101,65.098101,0.435538,0.81284,111.0,111.0,111.0
1,1,1.0,642133.1304,4.498932e+06,4444.0,111.0,11.0,4.445196,0.68200,1,...,NaN,NaN,65.098101,65.098101,65.098101,0.435538,0.81284,111.0,111.0,111.0
2,2,1.0,642134.7209,4.498937e+06,4444.0,111.0,11.0,7.459952,0.39028,1,...,NaN,NaN,65.098101,65.098101,65.098101,0.435538,0.81284,111.0,111.0,111.0
3,3,2.0,642115.6352,4.498880e+06,7777.0,666.0,77.0,10.083154,0.88016,1,...,NaN,NaN,65.098101,65.098101,65.098101,0.435538,0.81284,111.0,111.0,111.0
4,4,1.0,642117.2257,4.498885e+06,4444.0,111.0,11.0,10.088981,0.86376,1,...,NaN,NaN,65.098101,65.098101,65.098101,0.435538,0.81284,111.0,111.0,111.0


In [22]:
# Recode the values for CSV
pts_df4csv = pts_df.replace({'SubType': {7777:'{1111, 2222}', 1000:'{1111, 3333}'}, 
                              'VegType': {77:'{11, 22}', 88:'{22, 33}', 99:'{33, 44}'},
                              'VegDens': {666: '{111, 222}', 777: '{222, 333}', 
                                          888: '{333, 444}', 999: '{222, 333, 444}'}})

pts_df4csv.to_pickle(os.path.join(scratch_dir, transPts_null+'_4csv.pkl'))

pts_df4csv.sample(5)

,SplitSort,seg_x,seg_y,Dist_Seg,Dist_MHWbay,DistSegDH,DistSegDL,DistSegArm,ptZ,ptSlp,...,WidthLand,WidthFull,uBW,uBH,ub_feat,mean_Zmhw,max_Zmhw,Construction,Development,Nourishment
66615,66615,677018.1595,4.509975e+06,230.000032,82.220969,143.009575,149.259519,NaN,0.60388,2.552202,...,312.221001,312.221001,80.787403,3.090523,DL,0.957232,4.469680,111.0,111.0,222.0
85619,85619,687214.1370,4.514490e+06,345.696325,-131.024561,229.025318,240.263141,NaN,0.35092,0.961539,...,749.924166,855.620515,105.471719,4.944896,DL,0.591762,7.122921,111.0,111.0,222.0
66511,66511,677044.7910,4.509796e+06,55.000040,286.701757,32.017383,25.841729,NaN,1.81084,2.832443,...,341.701797,341.701797,80.572297,2.878211,DL,0.657066,3.999680,111.0,111.0,222.0
31938,31938,657490.3763,4.501733e+06,434.999996,137.250023,NaN,NaN,NaN,0.76025,NaN,...,572.250019,572.250019,NaN,NaN,NaN,NaN,3.859720,111.0,333.0,222.0
87919,87919,688190.4957,4.514639e+06,270.000020,275.964937,179.786747,188.534853,NaN,5.43132,6.142564,...,545.964957,545.964957,81.470031,3.533009,DL,1.878405,5.923080,111.0,111.0,222.0


In [15]:
# pID_fld needs to be among the columns
if not pID_fld in pts_df4csv.columns:
    pts_df4csv.reset_index(drop=False, inplace=True)

# Save CSV in scratch_dir
csv_fname = os.path.join(scratch_dir, transPts_fill +'.csv')
pts_df4csv.to_csv(os.path.join(scratch_dir, transPts_fill +'.csv'), na_rep=fill, index=False)
print("OUTPUT: {}".format(csv_fname))

try:
    xls_fname = os.path.join(scratch_dir, transPts_fill +'.xlsx')
    pts_df4csv.to_excel(xls_fname, na_rep=fill, index=False)
    print("OUTPUT: {}".format(xls_fname))
except:
    print("No Excel file created. You'll have to do it yourself from the CSV.")

OUTPUT: \\Mac\stor\Projects\TransectExtraction\FireIsland2012\scratch\FireIsland2012_transPts_fill.csv
No Excel file created. You'll have to do it yourself from the CSV.


In [18]:
tr_w_LRR = os.path.join(arcpy.env.workspace, 'fiis_trans_wLRR')
trans_df3 = fwa.FCtoDF(tr_w_LRR, id_fld=tID_fld, dffields=['LRR'])

# Load dataframes
trans_df= pd.read_pickle(os.path.join(scratch_dir, '20180108', extTrans_null+'_anthro.pkl'))
pts_df= pd.read_pickle(os.path.join(scratch_dir, '20180108', transPts_null+'_anthro.pkl'))

# Join anthro fields to trans and points DFs
trans_df = fun.join_columns(trans_df, trans_df3) 
pts_df = fun.join_columns(pts_df, trans_df, tID_fld)

# Save dataframes to open elsewhere or later
trans_df.to_pickle(os.path.join(scratch_dir, extTrans_null+'_anthroLRR.pkl'))
pts_df.to_pickle(os.path.join(scratch_dir, transPts_null+'_anthroLRR.pkl'))

Converting feature class to array...
Converting array to dataframe...


In [21]:
# Sort columns
pts_df = pts_df.reindex_axis(sorted_pt_flds, axis=1)

# Recode
pts_df4csv = pts_df.replace({'GeoSet': {9999:-99999},
                             'SubType': {7777:'{1111, 2222}', 1000:'{1111, 3333}', 9999:-99999}, 
                              'VegType': {77:'{11, 22}', 88:'{22, 33}', 99:'{33, 44}', 9999:-99999},
                              'VegDens': {666: '{111, 222}', 777: '{222, 333}', 
                                          888: '{333, 444}', 999: '{222, 333, 444}', 9999:-99999}})

# Save as pickle
pts_df4csv.to_pickle(os.path.join(scratch_dir, transPts_null+'_4csv.pkl'))

# Save as CSV
csv_fname = os.path.join(scratch_dir, transPts_fill +'.csv')
pts_df4csv.to_csv(os.path.join(scratch_dir, transPts_fill +'.csv'), na_rep=fill, index=False)
print("OUTPUT: {}".format(csv_fname))


OUTPUT: \\Mac\stor\Projects\TransectExtraction\FireIsland2012\scratch\FireIsland2012_transPts_fill.csv
